In [264]:
import chromedriver_binary
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
import time
import numpy as np
import pandas as pd
import concurrent.futures
import multiprocessing
import pprint
import re
import datetime
import locale

locale.setlocale(locale.LC_ALL, 'fr_FR.UTF-8')


'fr_FR.UTF-8'

In [283]:
df = {
    'nom' : [],
    'pays' : [],
    'favorite' : [],
    'date': [],
    'titre' : [],
    'bons_points' : [],
    'mauvais_points' : [],
    'note' : [],
    'type_etablissement' : [],
    'lieu' : [],
    'note_etablissement': []
}

In [271]:
def details_comment(df, review, etablissement):
    # review : Selenium object
    
    name = review.find_element_by_css_selector('.bui-avatar-block__title').text
    df['nom'].append(name)
    
    pays = review.find_element_by_css_selector('span.bui-avatar-block__subtitle').text
    df['pays'].append(pays)
    
    try:
        fav = review.find_element_by_css_selector('.c-review-block__badge').text
        df['favorite'].append(1)
    except NoSuchElementException:
        df['favorite'].append(0)
    
    # we get the sanitized date value
    date = review.find_element_by_css_selector('.c-review-block__date').text
    pattern = re.compile(r'\d{1,2}\s\w+\s\w{4}')
    result = pattern.search(date)
    date_str = result.group()
    date_format = "%d %B %Y"
    new_date = datetime.datetime.strptime(date_str, DATE_FORMAT)
    
    df['date'].append(new_date.date())
    
    titre = review.find_element_by_css_selector('.c-review-block__title').text
    df['titre'].append(titre)
    
    try:
        com_pos = review.find_element_by_css_selector('.c-review__inner:first-child .c-review__body').text
        df['bons_points'].append(com_pos)
    except NoSuchElementException:
        df['bons_points'].append('Pas de commentaires positifs')
    
    try:
        com_neg = review.find_element_by_css_selector('.lalala .c-review__body').text
        df['mauvais_points'].append(com_neg)
    except NoSuchElementException:
        df['mauvais_points'].append('Pas de commentaires négatifs')
        
    
    note = review.find_element_by_css_selector('.bui-review-score__badge').text
    df['note'].append(note)
    
    
    df['type_etablissement'].append(etablissement['type'])
    df['lieu'].append(etablissement['lieu'])
    df['note_etablissement'].append(etablissement['note'])

In [282]:
def get_comments(url):
    browser = webdriver.Chrome()
    
    # get page
    browser.get(url)
    
    # if are you humain popup appears, remove it
    try :
        btn_human = browser.find_element_by_css_selector('.bui-button bui-button--primary')
        btn_human.click()
    except NoSuchElementException:
         print('no popup to remove')
        
        
    # remove cookie banner because it takes almost 50% height of the page
    try:
        cookie_banner = browser.find_element_by_css_selector('#cookie_warning button')
        cookie_banner.click()
    except NoSuchElementException:
        print('no cookie banner to remove')
        

    # open reviews panel
    btn_cmt = browser.find_element_by_id('show_reviews_tab')
    btn_cmt.click()
    
    # get info about location
    etablissement = {
        'nom': browser.find_element_by_css_selector('.hp__hotel-name').text,
        'type' : browser.find_element_by_css_selector('.hp__hotel-name span').text,
        'note': browser.find_element_by_css_selector('.reviewFloater .bui-review-score__badge').get_attribute('innerHTML'), # sometimes it is hidden
        'lieu' : browser.find_element_by_css_selector('.sb-destination__input').get_attribute("value")
    }
    
    time.sleep(1)
        
    #get only french reviews
    btn_french = browser.find_element_by_css_selector('.language_filter .bui-input-checkbutton:first-child')
    btn_french.click()
    
    # it has to take into account the language change
    time.sleep(2)
    
    # get reviews_list
    reviews = browser.find_elements_by_css_selector('[itemprop="review"]')
    
    for i in range(len(reviews)): 
        reviews = browser.find_elements_by_css_selector('[itemprop="review"]')
        review =  reviews[i]
        
        browser.execute_script('arguments[0].scrollIntoView({behavior: "smooth", block: "end", inline: "nearest"});', review)
        
        # collect details about the review and store it inside the dictionary
        details_comment(df, review, etablissement)
        time.sleep(2)
    
    close_btn = browser.find_element_by_css_selector('.sliding-panel-widget.is-shown .sliding-panel-widget-close-button')
    close_btn.click()
        
    # ========= DEBUG ONLY ============= #
    # pp = pprint.PrettyPrinter(indent=4)
    # pp.pprint(df)
    # ========= END DEBUG ONLY ========== #
    
    # save data collected and save it to a CSV file
    data = pd.DataFrame(df)
    data.to_csv('booking.csv', index=False)
    return data

In [284]:
url = "https://www.booking.com/hotel/fr/appartement-vue-sur-mer-fleury.fr.html?aid=304142;label=gen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4Au6WtPYFwAIB;sid=1f08e1d7b9a399a0cf3d0b1ac8c1b4d3;all_sr_blocks=402947801_264223935_1_0_0;checkin=2020-06-03;checkout=2020-06-06;dest_id=73;dest_type=country;dist=0;group_adults=1;group_children=0;hapos=976;highlighted_blocks=402947801_264223935_1_0_0;hpos=1;no_rooms=1;req_adults=1;req_children=0;room1=A;sb_price_type=total;sr_order=upsort_bh;sr_pri_blocks=402947801_264223935_1_0_0__22574;srepoch=1590496226;srpvid=689f57f04b2d004a;type=total;ucfs=1&#tab-main"
get_comments(url)

no popup to remove

<i class="bicon-aclose" aria-hidden="true"></i>



,nom,pays,favorite,date,titre,bons_points,mauvais_points,note,type_etablissement,lieu,note_etablissement
0,Sandrine,France,1,2018-10-19,Très satisfaits de notre séjour! Adresse à ret...,La personne qui nous accueille (Nadine) est ex...,Il faudrait éventuellement envoyer un plan d'a...,10,Appartement,Fleury,"7,9"
1,Anne,France,0,2019-06-10,Appartement très bien placé!,"Nous cinq était accueilli par Nadine, très arr...",Pas de commentaires négatifs,"8,8",Appartement,Fleury,"7,9"
2,Valérie,France,0,2019-10-11,Bon,La vue sur mer.,Payer 40 euros pour le ménage de sortie \n... ...,"5,8",Appartement,Fleury,"7,9"
3,Samantha,France,0,2019-09-01,Bon week-end,"Les équipements, la proximité et l’accueil",Pas de commentaires négatifs,"9,6",Appartement,Fleury,"7,9"
4,Jean-jacques,France,0,2018-10-18,Idéal pour un cour séjour.,Suite aux intempéries l'eau s'infiltrait par l...,Suite aux intempéries l'eau s'infiltrait par l...,"6,2",Appartement,Fleury,"7,9"
5,Simon,France,0,2019-11-15,Bien,Ce client n'a pas laissé de commentaire.,Pas de commentaires négatifs,"7,9",Appartement,Fleury,"7,9"
